In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import random
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

import requests
import json


client_id = 'dc2f1fa6-fb4e-488f-a13e-f2ad8c1edecd'


In [ ]:
#Temperatures for: DRAMMEN - BANGELØKKA
endpoint = 'https://frost.met.no/observations/v0.jsonld?sources=SN26580&referencetime=2021-02-01/2021-02-10&elements=air_temperature&timeoffsets=default&levels=default'


# Issue an HTTP GET request
r = requests.get(endpoint, auth=(client_id,''))
# Extract JSON data
result_json = r.json()
data = result_json['data']

In [ ]:
data

In [ ]:
#Retrieving the data of interest by looping through every entry of the list
data_tmp = [(entry['referenceTime'], obs['value']) for entry in data for obs in entry['observations']]

#Adding the data_tmp-values to a dataframe called temperature_df
temperature_df = pd.DataFrame(data_tmp, columns=['referenceTime', 'value'])
#Renaming the 'value'-column to 'historical_temperature'
temperature_df.rename(columns={'value':'historical_temperatures'}, inplace=True)

#Making the 'reference_time the datetime index of the dataframe
temperature_df['referenceTime'] = pd.to_datetime(temperature_df['referenceTime'])
temperature_df.set_index('referenceTime', inplace=True)
#Resampling to hourly resolution and interpolating
temperature_df = temperature_df.resample('H').interpolate()


In [ ]:
temperature_df.head(2)

In [ ]:
#Plotting the temperatures

fig = px.line(temperature_df, title='Temperature plot')

# Show the plot
fig.show()


In [ ]:
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf

training_start_date = '2021-02-01 00:00:00+00:00'
training_end_date = '2021-02-01 23:00:00+00:00'
predicting_start_date = '2021-02-02 00:00:00+00:00'
predicting_end_date = '2021-02-02 23:00:00+00:00'

# Filter data within the specified datetime range
df_train = temperature_df.loc[training_start_date:training_end_date]
df_test = temperature_df.loc[predicting_start_date:predicting_end_date]


In [ ]:
#Plotting the temperatures

fig = px.line(df_test, title='Temperature plot')

# Show the plot
fig.show()


In [ ]:
from statsmodels.tsa.stattools import adfuller
adf_test = adfuller(df_train)
print(f'p-value: {adf_test[1]}')

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

def predicting(df_train, df_test, start_date, end_date, p, d, q):
    model = ARIMA(df_train, order=(p,d,q))
    model = model.fit()
    df_test['predictions'] = model.predict(start=start_date, end=end_date, typ='levels')
    df_test[start_date:end_date]

In [ ]:
from statsmodels.graphics.tsaplots import plot_pacf
plot_pacf(df_test['historical_temperatures'].diff().dropna())

In [ ]:
plot_acf(df_test['historical_temperatures'].diff().dropna())

In [ ]:
start_date='2021-02-02 00:00:00+00:00'
end_date='2021-02-02 23:00:00+00:00'
p=6
d=1
q=1
#p = number of past values included in the AR model
#d = number of times the time series is differenced
#q = the number of past forecast errors included in the MA model

predicting(df_test=df_test, df_train=df_train, start_date=start_date,end_date=end_date, p=p, d=d, q=q)


In [ ]:
prediction_fig = px.line()

prediction_fig.add_trace(
                go.Scatter(
                    x=temperature_df.index,
                    y=temperature_df['historical_temperatures']['2021-02-01 00:00:00+00:00':'2021-02-03 09:00:00+00:00'],
                    mode="lines",
                    name='actual historical temperatures',
                    line=dict(color="blue"),
                )
            )
prediction_fig.add_trace(
                go.Scatter(
                    x=df_test.index,
                    y=df_test['predictions'],
                    mode="lines",
                    name='Predictions',
                    line=dict(color="green"),
                )
            )
prediction_fig.show()


In [ ]:

difference = df_test['predictions'] - df_test['historical_temperatures']
difference_plot = px.line(df_test['predictions'] - df_test['historical_temperatures'])
difference_plot.show()

In [ ]:
import pmdarima as pm
auto_arima = pm.auto_arima(df_train, stepwise=False, seasonal=False)
auto_arima

In [ ]:
auto_arima.summary()

In [ ]:
forecast_test_auto = auto_arima.predict(n_periods=len(df_test))

df = pd.DataFrame()
df['forecast_auto'] = [None]*len(df_train) + list(forecast_test_auto)
df.plot()

## Monte Carlo Simulation

In [ ]:
import numpy as np

def monte_carlo_simulation(df_train, df_test, start_date, end_date, p, d, q, num_simulations):

    montecarlofig = px.line()

    for i in range(num_simulations):


        p_variation = np.random.randint(-1, 2) 
        d_variation = np.random.randint(-1, 2)
        q_variation = np.random.randint(-1, 2)
    
        model = ARIMA(df_train, order=(p + p_variation, d + d_variation, q + q_variation))
        model = model.fit()

        df_test[f'Simulation_{i + 1}'] = model.predict(start=start_date, end=end_date, typ='levels')


        montecarlofig.add_trace(
                go.Scatter(
                    x=df_test.index,
                    y=df_test[f'Simulation_{i + 1}'],
                    mode="lines",
                    name=f'Simulation_{i + 1}',
                    line=dict(color="green"),
                )
            )
        
    montecarlofig.add_trace(
    go.Scatter(
        x=df_test.index,
        y=df_test['predictions'],
        mode="lines",
        name=f'Initial prediction',
        line=dict(color="red"),
        )
    )
    montecarlofig.add_trace(
    go.Scatter(
        x=df_test.index,
        y=df_test['historical_temperatures'],
        mode="lines",
        name=f'Historical data',
        line=dict(color="blue"),
        )
    )
        

    montecarlofig.show()

In [ ]:
monte_carlo_simulation(df_train, df_test, start_date, end_date, p=1, d=1, q=1, num_simulations=1000)